<a href="https://www.kaggle.com/code/andreisavchenko/baseline-v1?scriptVersionId=137141007" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [ ]:
# Извлечем из адреса отеля страну
def country_func(hotel_address):
    hotel_address = hotel_address.split()
    res = hotel_address[-2:]
    for i in res:
        if i == 'United':
            return 'United Kingdom'
        else:
            return ''.join(res[-1:])
    
data['hotel_country'] = data['hotel_address'].apply(country_func)

In [ ]:
data.hotel_country.unique()

In [ ]:
# Создадим нрвый признак который покажет, человек который оставил отзыв приехал с другой страны или нет
data['reviewer_nationality'] = data['reviewer_nationality'].map(str.strip)
data['reviewer_tourist'] = data['hotel_address'] != data['reviewer_nationality']

In [ ]:
# Преобразуем признак с тегами, чтоб удобнее было с ним работать
def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

data['tags_n'] = data['tags'].apply(rev_func)

In [ ]:
# Используя теги выясним была ли поездка деловой или это был отдых
def leisure_func(tags_n):
    for i in tags_n:
        if i == 'Business trip':
            return 0
        else:
            return 1
    
data['leisure_trip'] = data['tags_n'].apply(leisure_func)

def business_func(tags_n):
    for i in tags_n:
        if i == 'Business trip':
            return 1
        else:
            return 0
    
data['business_trip'] = data['tags_n'].apply(business_func)

In [ ]:
# Также из тегов постараемся выяснить был ли путешественик один
def solo_func(tags_n):
    for i in tags_n:
        if i != 'Solo traveler':
            continue
        else:
            return 1
    return 0
    
data['solo_traveler'] = data['tags_n'].apply(solo_func)

In [ ]:
# Из тегов выясним на сколько ночей остонавливались в отеле
def night_func(tags_n):
    for i in tags_n:
        if 'night' not in i:
            continue
        else:
            return i[7]
    return 0
    
data['night'] = data['tags_n'].apply(night_func)

In [ ]:
# Используем SentimentIntensityAnalyzer для оценки отзывов
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
nltk.downloader.download('vader_lexicon')

def sentiment_func(review):
    
    # Create a SentimentIntensityAnalyzer object.
    sent_analyzer = SentimentIntensityAnalyzer()
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sent_analyzer.polarity_scores(review)
    return sentiment_dict

In [ ]:
# Для более адекватной оценки SentimentIntensityAnalyzer отзывов заменим следующие значения
data['negative_review'] = data['negative_review']. replace ('No Negative', 'Positive')
data['positive_review'] = data['positive_review']. replace ('No Positive', 'Negative')

In [ ]:
data['sentiment_n'] = data['negative_review'].apply(sentiment_func)

In [ ]:
data['sentiment_p'] = data['positive_review'].apply(sentiment_func)

In [ ]:
data.head(3)

In [ ]:
def sent_neg_n(sentiment_n):
    res = sentiment_n['neg']
    return res

data['neg_n'] = data['sentiment_n'].apply(sent_neg_n)

def sent_neu_n(sentiment_n):
    res = sentiment_n['neu']
    return res

data['neu_n'] = data['sentiment_n'].apply(sent_neu_n)

def sent_pos_n(sentiment_n):
    res = sentiment_n['pos']
    return res

data['pos_n'] = data['sentiment_n'].apply(sent_pos_n)

def sent_compound_n(sentiment_n):
    res = sentiment_n['compound']
    return res

data['compound_n'] = data['sentiment_n'].apply(sent_compound_n)

In [ ]:
def sent_neg_p(sentiment_p):
    res = sentiment_p['neg']
    return res

data['neg_p'] = data['sentiment_p'].apply(sent_neg_p)

def sent_neu_p(sentiment_p):
    res = sentiment_p['neu']
    return res

data['neu_p'] = data['sentiment_p'].apply(sent_neu_p)

def sent_pos_p(sentiment_p):
    res = sentiment_p['pos']
    return res

data['pos_p'] = data['sentiment_p'].apply(sent_pos_p)

def sent_compound_p(sentiment_p):
    res = sentiment_p['compound']
    return res

data['compound_p'] = data['sentiment_p'].apply(sent_compound_p)

In [ ]:
# Преобразуем признак 'reviewer_tourist'
data = pd.get_dummies(data, columns=['reviewer_tourist'])

In [ ]:
data['night'] = pd.to_numeric(data['night'], errors='coerce').fillna(0)

In [ ]:
data.info()

In [ ]:
# По моему мнению широта и долгота слабо влияют на оценку отеля, поэтому просто удаляем их (место расположения отеля видно из адреса)
data.drop(['lat', 'lng'], axis=1, inplace=True)
data.info()

In [ ]:
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
object_columns

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# зададим параметры холста, название и визуализируем кривые распределения:
fig, (ax1) = plt.subplots(ncols=1, figsize=(14, 12))
ax1.set_title('Распределения')

# kdeplot() (KDE – оценка плотности ядра) – специальный метод для графиков распределений
sns.kdeplot(data['review_total_positive_word_counts'], ax=ax1, label ='review_total_positive_word_counts')
sns.kdeplot(data['review_total_negative_word_counts'], ax=ax1, label ='review_total_negative_word_counts')
plt.legend()

In [ ]:
fig, (ax1) = plt.subplots(ncols=1, figsize=(14, 12))
ax1.set_title('Распределения')

sns.kdeplot(data['neg_n'], ax=ax1, label ='neg_n')
sns.kdeplot(data['neu_n'], ax=ax1, label ='neu_n')
sns.kdeplot(data['pos_n'], ax=ax1, label ='pos_n')
sns.kdeplot(data['compound_n'], ax=ax1, label ='compound_n')
plt.legend()

In [ ]:
fig, (ax1) = plt.subplots(ncols=1, figsize=(14, 12))
ax1.set_title('Распределения')

sns.kdeplot(data['neg_p'], ax=ax1, label ='neg_p')
sns.kdeplot(data['neu_p'], ax=ax1, label ='neu_p')
sns.kdeplot(data['pos_p'], ax=ax1, label ='pos_p')
sns.kdeplot(data['compound_p'], ax=ax1, label ='compound_p')
plt.legend()

In [ ]:
data_s = data[['additional_number_of_scoring',
 'review_total_negative_word_counts',
 'total_number_of_reviews',
 'review_total_positive_word_counts',
 'total_number_of_reviews_reviewer_has_given',
 'leisure_trip',
 'business_trip',
 'solo_traveler',
 'night',
 'neg_n',
 'neu_n',
 'pos_n',
 'compound_n',
 'neg_p',
 'neu_p',
 'pos_p',
 'compound_p',
 'average_score',
 'reviewer_tourist_True']]
data_s.info()

In [ ]:
target = data[['reviewer_score', 'sample']]
target.info()

In [ ]:
# Копируем названия столбцов, которые теряются при использовании fit_transform()
col_names = list(data_s.columns)

In [ ]:
# для нормализации, стандартизации
from sklearn import preprocessing
# инициализируем нормализатор MinMaxScaler
scaler = preprocessing.MinMaxScaler()

# кодируем исходный датасет
data_sc = scaler.fit_transform(data_s)

# Преобразуем промежуточный датасет в полноценный датафрейм для визуализации
data_mms = pd.DataFrame(data_sc, columns=col_names)

In [ ]:
plt.rcParams['figure.figsize'] = (18,14)
sns.heatmap((data_mms).corr(), annot=True)

Удалим признаки с очень сильной корреляцией (где коэффициент корреляции +/-0.7 и выше):

pos_p + compound_p; neu_n + pos_n; leisure_trip + business_trip; total_number_of_reviews + additional_number_of_scoring; compound_n + pos_n

In [ ]:
data_mms.drop(['compound_p', 'pos_n', 'business_trip', 'additional_number_of_scoring'], axis=1, inplace=True)

In [ ]:
plt.rcParams['figure.figsize'] = (18,14)
sns.heatmap((data_mms).corr(), annot=True)

In [ ]:
data_f = pd.concat([
    data_mms,
    target
], axis=1)
data_f.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data_f.query('sample == 1').drop(['sample'], axis=1)
test_data = data_f.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# непрерывные признаки
num_cols = ['total_number_of_reviews', 'review_total_negative_word_counts', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'night', 'neg_n', 'neu_n', 'compound_n', 'neg_p', 'neu_p', 'pos_p']

# категориальные признаки
cat_cols = ['average_score', 'reviewer_tourist_True', 'solo_traveler', 'leisure_trip']

In [ ]:
y=y.astype('int')

...
from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# удалим менее значимые признаки
X = X.drop(['night', 'neu_p', 'total_number_of_reviews', 'total_number_of_reviews_reviewer_has_given', 'reviewer_tourist_True'], axis = 1)

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest().plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)